In [1]:
import os
from tqdm import tqdm
import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
from joblib import Parallel, delayed

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)
        
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [3]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
0,578,2013-01-02,1,103665,2.0,Unk,BREAD/BAKERY,2712,1,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_103665
1,579,2013-01-02,1,105574,8.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105574
2,580,2013-01-02,1,105575,15.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105575
3,581,2013-01-02,1,105577,2.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105577
4,582,2013-01-02,1,105737,2.0,Unk,GROCERY I,1044,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105737


In [5]:
df.shape

(2562153, 27)

In [6]:
len(set(df.item_nbr))

3567

In [7]:
np.min(df.date)

'2013-01-02'

In [8]:
np.max(df.date)

'2017-08-15'

In [9]:
np.datetime64(np.max(df.date)) - np.datetime64(np.min(df.date))

numpy.timedelta64(1686,'D')

In [10]:
365 * 4

1460

In [11]:
def set_recent_stats(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_mean'.format(days)] = mean
    df2.loc[0,'us_{}_med'.format(days)] = median
    df2.loc[0,'us_{}_std'.format(days)] = std
    df2.loc[0,'us_{}_skew'.format(days)] = skew
    df2.loc[0,'us_{}_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo2(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo3(df2, date, days, fn, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety

def set_recent_stats_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_h_mean'.format(days)] = mean
    df2.loc[0,'us_{}_h_med'.format(days)] = median
    df2.loc[0,'us_{}_h_std'.format(days)] = std
    df2.loc[0,'us_{}_h_skew'.format(days)] = skew
    df2.loc[0,'us_{}_h_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_h_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_per(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_per_mean'.format(days)] = mean
    df2.loc[0,'us_{}_per_med'.format(days)] = median
    df2.loc[0,'us_{}_per_std'.format(days)] = std
    df2.loc[0,'us_{}_per_skew'.format(days)] = skew
    df2.loc[0,'us_{}_per_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_per_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_nper(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_nper_mean'.format(days)] = mean
    df2.loc[0,'us_{}_nper_med'.format(days)] = median
    df2.loc[0,'us_{}_nper_std'.format(days)] = std
    df2.loc[0,'us_{}_nper_skew'.format(days)] = skew
    df2.loc[0,'us_{}_nper_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_nper_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow(df2, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_tbn'.format(days, dow)] = ten_by_ninety

def set_recent_stats_promo_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_ph_mean'.format(days)] = mean
    df2.loc[0, 'us_{}_ph_med'.format(days)] = median
    df2.loc[0, 'us_{}_ph_std'.format(days)] = std
    df2.loc[0, 'us_{}_ph_skew'.format(days)] = skew
    df2.loc[0, 'us_{}_ph_kurt'.format(days)] = kurtosis
    df2.loc[0, 'us_{}_ph_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow_promo_hol(df1, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_ph_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_ph_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_ph_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_ph_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_ph_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_ph_tbn'.format(days, dow)] = ten_by_ninety


In [14]:
# df_h = pd.read_csv('../data/holidays_events.csv')

In [15]:
# df_h.type.value_counts()

In [16]:
# df_h1 = df_h[(df.date.apply(np.datetime64)) > np.datetime64('2017-08-15')]

In [17]:
df.onpromotion.value_counts()

False    1932613
Unk       484853
True      138020
False       6667
Name: onpromotion, dtype: int64

In [18]:
df.onpromotion[df.onpromotion==True].value_counts()

True    138020
Name: onpromotion, dtype: int64

In [19]:
# df.h_type.value_counts()

In [20]:
# df.h_type[df.h_type=='Work Day'].value_counts()

In [21]:
# df.h_type[df.h_type != 'Work Day'].value_counts()

In [22]:
#df_h.h_type[df_h.type == 'Additional'] = 'Holiday'

In [23]:
recent_days = [30, 90, 180, 365, 730]

In [24]:
def calculate_recent_stats(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        

def calculate_recent_stats_promo(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo3(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
        
def calculate_recent_stats_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.h_type != 'Work Day')]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
 
def calculate_recent_stats_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 1)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_per(df3, date,days, mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_not_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 0)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_nper(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_dow(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_dow(df3, date,days, dow, 0, 0, 0, 0, 0, 0)
            continue
        
#         data = df.unit_sales[(df.dts < np.datetime64(date)) & 
#                              (df.dts >= past_date) & (df.dow == dow)]
        
        data = df.unit_sales[(df.date.astype('str') < str(date)) & 
                             (df.date.astype('str') >= str(past_date)) & (df.dow == dow) ]
        
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_dow(df3, date,days,dow, 0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow(df3, date,days, dow, mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_promo_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_dow_promo_hol(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & 
                             (df.onpromotion == True) & 
                             (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,dow,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow_promo_hol(df3, date,days,dow,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3

In [25]:
print(df.unit_sales.mean())

5.52075222249


In [26]:
past_date = (np.datetime64('2016-01-01') - np.timedelta64(2, 'D'))

In [27]:
# df.unit_sales[(df.date.apply(np.datetime64) >= past_date) & 
#               (df.date.apply(np.datetime64) < np.datetime64('2016-01-01'))].mean()

In [28]:
df1 = df.set_index('date')

In [29]:
s1 = df1.loc['2015-12-31', 'unit_sales']
s2 = df1.loc['2015-12-30', 'unit_sales']

In [30]:
s = pd.concat([s1, s2], axis=0)

In [31]:
s.mean()

6.5914254756871031

In [32]:
gc.enable()

In [33]:
# import os
# x = {i for i in range(20)}
# os.sched_setaffinity(0, x)

In [34]:
# chunks = []
# for i in range(1, len(d), 1000):
#    chunks.append(d[i:min(i + 1000, len(d))])

# res = Parallel(n_jobs=-1) (delayed(hash) (d) for d in chunks)

# def hash(d):
#     return [hashlib.sha256(str(x).encode('utf-8')).hexdigest()[:30] for x in d.itertuples(index=False, name=None)]

In [35]:
# Parallel(n_jobs=20)(delayed(sqrt)(i ** 2) for i in range(10000000))

In [36]:
# df = pd.DataFrame()
# df1 = pd.DataFrame()

In [37]:
# for i in range(1,55):
#     print(i)
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     df2 = pd.read_csv(fn)
#     df2.unit_sales[df2.unit_sales < 0] = 0
#     df3 = df2.set_index('date')
#     dates = np.unique(df2.date.values)
# #     for j, date in tqdm(enumerate(dates)):
#     df = df2.copy()
#     df1 = df3.copy()
#     df = df.iloc[::-1]
#     Parallel(n_jobs=10)(delayed(calculate_recent_stats)(date) for date in tqdm(dates))
# #         calculate_recent_stats(date, df, df1)
#     for days in [30, 90, 180, 365, 730]:
#         df = set_recent_df_stats(df ,df1, days)
#     df = df.iloc[::-1]
#     print(df.shape)
#     print(sum(df.isnull().any()==True)) # print NA
#     print(sum((df != 0).any(axis=0))) # print non-zero columns
#     fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
#     print(fn)
#     df.to_csv(fn, index=False)
#     del df2
#     del df3

In [38]:
# files_list = []
# for i in range(1,55):
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     files_list.append(fn)

In [39]:
# print('creating pool..')
# print(cpu_count())
# thread_pool = Pool(10)
# print('mapping pool..')
# thread_pool.map (calculate_recent_stats2, arg_list)

# print('joining pool..')

In [40]:
# from multiprocessing import Pool, cpu_count
def process_csv_file2(i):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn)
    df = pd.read_csv(fn)
    print(df.shape)
    df.unit_sales[df.unit_sales < 0] = 0 
    df = df.iloc[::-1]
    df1 = df.set_index('date')

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    for date in tqdm(trend_dates):
        calculate_recent_stats_not_perishable(date, df, df1)
    for days in [30, 90, 180, 365, 730]:
        df = set_recent_df_stats_nper(df, df1, days)
    df = df.iloc[::-1]
    
    df = df[(df.date.apply(np.datetime64) >= np.datetime64('2015-01-01')) & 
              (df.date.apply(np.datetime64) <= np.datetime64('2017-08-15'))]
    
    print(df.shape)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_nper.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    del df
    del df1

In [41]:
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [43]:
# x = {i for i in range(20)}
# os.sched_setaffinity(0, x)

In [44]:
def process_csv_file1(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
    df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    for dow in range(7):
        fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        delete_file_if_exists(fn_tmp)
        for date in tqdm(trend_dates):
            calculate_recent_stats_dow(date, dow, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    for dow in range(7):
        fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        df1 = pd.read_csv(fn_tmp)
        print(df1.shape)
        df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_dow.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1

In [45]:
def process_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    for dow in range(7):
        fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        delete_file_if_exists(fn_tmp)
        for date in tqdm(trend_dates):
            calculate_recent_stats_dow(date, dow, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    for dow in range(7):
        fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        df1 = pd.read_csv(fn_tmp)
        print(df1.shape)
        df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_dow_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f',index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    for dow in range(7):
        fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        delete_file_if_exists(fn_tmp)

In [48]:
def process_test_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(i)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2017-08-16')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    for dow in range(7):
        fn_tmp = '../cache/test7_1_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        delete_file_if_exists(fn_tmp)
        for date in tqdm(trend_dates):
            calculate_recent_stats_dow(date, dow, df, fn_tmp)

    print(df.shape)
#     df = df[(df.dts >= np.datetime64('2017-08-16')) & (df.dts <= np.datetime64('2017-08-31'))]
    df = df[(df.date.astype('str') >= str('2017-08-16')) & (df.date.astype('str') <= str('2017-08-31'))]

    print(df.shape)
    nrows1 = df.shape[0]
    for dow in range(7):
        fn_tmp = '../cache/test7_1_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        df1 = pd.read_csv(fn_tmp)
        print(df1.shape)
        df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/test7_1_t_store{}_recent_stats_dow_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f',index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    for dow in range(7):
        fn_tmp = '../cache/test7_1_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
        delete_file_if_exists(fn_tmp)

In [49]:
# for i in range(1,55):
#     process_test_csv_file(i) 

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 16/16 [01:36<00:00,  6.05s/it]


(2624569, 28)
(62416, 28)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
0
236
../cache/test7_1_t_store1_recent_stats_dow_1.csv
(62416, 237)
../cache/bkup2/test7_1_t_store2.csv


100%|██████████| 16/16 [01:52<00:00,  7.01s/it]


(3050256, 28)
(62416, 28)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
0
236
../cache/test7_1_t_store2_recent_stats_dow_1.csv
(62416, 237)
../cache/bkup2/test7_1_t_store3.csv


...

(62416, 28)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
0
110
../cache/test7_1_t_store52_recent_stats_dow_1.csv
(62416, 237)
../cache/bkup2/test7_1_t_store53.csv


100%|██████████| 16/16 [01:13<00:00,  4.60s/it]


(2000671, 28)
(62416, 28)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
0
236
../cache/test7_1_t_store53_recent_stats_dow_1.csv
(62416, 237)
../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 16/16 [01:03<00:00,  3.95s/it]


(1711283, 28)
(62416, 28)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
(16, 31)
0
236
../cache/test7_1_t_store54_recent_stats_dow_1.csv
(62416, 237)


In [3]:
date_block = {}
date_block['1_2013'] = 0
date_block['2_2013'] = 1
date_block['3_2013'] = 2
date_block['4_2013'] = 3
date_block['5_2013'] = 4
date_block['6_2013'] = 5
date_block['7_2013'] = 6
date_block['8_2013'] = 7
date_block['9_2013'] = 8
date_block['10_2013'] = 9
date_block['11_2013'] = 10
date_block['12_2013'] = 11

date_block['1_2014'] = 12
date_block['2_2014'] = 13
date_block['3_2014'] = 14
date_block['4_2014'] = 15
date_block['5_2014'] = 16
date_block['6_2014'] = 17
date_block['7_2014'] = 18
date_block['8_2014'] = 19
date_block['9_2014'] = 20
date_block['10_2014'] = 21
date_block['11_2014'] = 22
date_block['12_2014'] = 23

date_block['1_2015'] = 24
date_block['2_2015'] = 25
date_block['3_2015'] = 26
date_block['4_2015'] = 27
date_block['5_2015'] = 28
date_block['6_2015'] = 29
date_block['7_2015'] = 30
date_block['8_2015'] = 31
date_block['9_2015'] = 32
date_block['10_2015'] = 33
date_block['11_2015'] = 34
date_block['12_2015'] = 35

date_block['1_2016'] = 36
date_block['2_2016'] = 37
date_block['3_2016'] = 38
date_block['4_2016'] = 39
date_block['5_2016'] = 40
date_block['6_2016'] = 41
date_block['7_2016'] = 42
date_block['8_2016'] = 43
date_block['9_2016'] = 44
date_block['10_2016'] = 45
date_block['11_2016'] = 46
date_block['12_2016'] = 47

date_block['1_2017'] = 48
date_block['2_2017'] = 49
date_block['3_2017'] = 50
date_block['4_2017'] = 51
date_block['5_2017'] = 52
date_block['6_2017'] = 53
date_block['7_2017'] = 54
date_block['8_2017'] = 55
date_block['9_2017'] = 56
date_block['10_2017'] = 57
date_block['11_2017'] = 58
date_block['12_2017'] = 59

def get_date_block_num(x):
    year, month, day = x.split('-')
    mon = int(month)
    yr = int(year)
    mon_year = str(mon) + '_' + str(yr)
    
    return date_block[mon_year]

In [4]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
    print(df.shape)
    df['date_block_num']  = df.date.map(lambda x: get_date_block_num(x))
    df['in_dow']  = df.item_nbr.astype('str') + '_' + df.dow.astype('str')
    
#     df = df.drop_duplicates()
    unit_sales_dow_mean = df.groupby(['in_dow', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_dow_mean':'mean'}, 
                                                         })
    unit_sales_dow_mean.columns = ['in_dow', 'date_block_num', 'unit_sales_dow_mean']
    df = pd.merge(df, unit_sales_dow_mean, how='left', on=['in_dow', 'date_block_num']).fillna(0.0)

#     unit_sales_median = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_median':'median'}, 
#                                                          })
#     unit_sales_median.columns = ['item_nbr', 'date_block_num', 'unit_sales_median']
#     df = pd.merge(df, unit_sales_median, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)
    
#     unit_sales_std = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_std':'std'}, 
#                                                          })
#     unit_sales_std.columns = ['item_nbr', 'date_block_num', 'unit_sales_std']
#     df = pd.merge(df, unit_sales_std, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)
    
    
    shifted_df = df[['id', 'date_block_num','in_dow','unit_sales_dow_mean']].copy()
    
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
    shifted_df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    del unit_sales_dow_mean
#     del unit_sales_hol_median
#     del unit_sales_hol_std


  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2624569, 27)


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/pandas/core/groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
  2%|▏         | 1/54 [00:32<29:00, 32.84s/it]

...

 96%|█████████▋| 52/54 [25:48<00:59, 29.78s/it]

../cache/bkup2/test7_1_t_store53.csv
(2000671, 27)


 98%|█████████▊| 53/54 [26:12<00:29, 29.67s/it]

../cache/bkup2/test7_1_t_store54.csv
(1711283, 27)


100%|██████████| 54/54 [26:33<00:00, 29.51s/it]


In [5]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','in_dow','unit_sales_dow_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 1
    shifted_df.columns = ['date_block_num','in_dow', 'unit_sales_dow_mean_f_lag_1']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','in_dow'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_dow_mean_f_lag_1'] = df1.unit_sales_dow_mean_f_lag_1.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag1_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

100%|██████████| 54/54 [08:52<00:00,  9.87s/it]


In [6]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','in_dow','unit_sales_dow_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 3
    shifted_df.columns = ['date_block_num','in_dow', 'unit_sales_dow_mean_f_lag_3']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','in_dow'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_dow_mean_f_lag_3'] = df1.unit_sales_dow_mean_f_lag_3.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag3_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

100%|██████████| 54/54 [08:47<00:00,  9.76s/it]


In [7]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','in_dow','unit_sales_dow_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 6
    shifted_df.columns = ['date_block_num','in_dow', 'unit_sales_dow_mean_f_lag_6']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','in_dow'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_dow_mean_f_lag_6'] = df1.unit_sales_dow_mean_f_lag_6.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag6_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [08:24<00:29,  9.90s/it]

52
(352997, 5)
{51, 52, 53, 54, 55}
5
{57, 58, 59, 60, 61}
(352997, 1)
------


100%|██████████| 54/54 [08:40<00:00,  9.64s/it]


In [8]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','in_dow','unit_sales_dow_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 12
    shifted_df.columns = ['date_block_num','in_dow', 'unit_sales_dow_mean_f_lag_12']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','in_dow'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_dow_mean_f_lag_12'] = df1.unit_sales_dow_mean_f_lag_12.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag12_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [08:13<00:29,  9.68s/it]

52
(352997, 5)
{51, 52, 53, 54, 55}
5
{64, 65, 66, 67, 63}
(352997, 1)
------


100%|██████████| 54/54 [08:28<00:00,  9.43s/it]


In [9]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsdow_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','in_dow','unit_sales_dow_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 24
    shifted_df.columns = ['date_block_num','in_dow', 'unit_sales_dow_mean_f_lag_24']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','in_dow'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_dow_mean_f_lag_24'] = df1.unit_sales_dow_mean_f_lag_24.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag24_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 39%|███▉      | 21/54 [03:10<04:58,  9.06s/it]

22
(985899, 5)
{33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55}
23
{57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
(985899, 1)
------


 94%|█████████▍| 51/54 [07:49<00:27,  9.20s/it]

52
(352997, 5)
{51, 52, 53, 54, 55}
5
{75, 76, 77, 78, 79}
(352997, 1)
------


100%|██████████| 54/54 [08:03<00:00,  8.96s/it]


In [10]:
for shop_num in tqdm(range(1,55)):
    df1 = pd.DataFrame()
    
    df1['unit_sales_dow_mean_f_lag_1'] =0
    df1['unit_sales_dow_mean_f_lag_3'] = 0
    df1['unit_sales_dow_mean_f_lag_6'] = 0
    df1['unit_sales_dow_mean_f_lag_12'] = 0
    df1['unit_sales_dow_mean_f_lag_24'] = 0
    
    fn = '../cache/lag1_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    df2 = pd.read_csv(fn)
    
    fn = '../cache/lag3_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    df3 = pd.read_csv(fn)
    
    fn = '../cache/lag6_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    df4 = pd.read_csv(fn)
    
    fn = '../cache/lag12_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    df5 = pd.read_csv(fn)
    
    fn = '../cache/lag24_sales_mean_rsdow_df_shop{}.csv'.format(shop_num)
    df6 = pd.read_csv(fn)
    
    df1['id'] = df2['id'].values
    if len(df2.columns) > 1:
        df1['unit_sales_dow_mean_f_lag_1'] = df2.unit_sales_dow_mean_f_lag_1.values
        
    if len(df3.columns) > 1:
        df1['unit_sales_dow_mean_f_lag_3'] = df3.unit_sales_dow_mean_f_lag_3.values
        
        
    if len(df4.columns) > 1:
        df1['unit_sales_dow_mean_f_lag_6'] = df4.unit_sales_dow_mean_f_lag_6.values
        
    if len(df5.columns) > 1:
        df1['unit_sales_dow_mean_f_lag_12'] = df5.unit_sales_dow_mean_f_lag_12.values
        
    if len(df6.columns) > 1:
        df1['unit_sales_dow_mean_f_lag_24'] = df6.unit_sales_dow_mean_f_lag_24.values
     
    
    
    fn = '../cache/df_all_rsdow_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
#     const_col = []
#     for col in df1.columns:
#         c = len(set(df1[col].values))
#         if c==1:
#             const_col.append(col)
#     if len(const_col) > 0:
#         c = len(const_col)
#         print(shop_num)
#         print(c)
#         print(df1.shape)
#         print(const_col)
    df1.to_csv(fn, float_format='%.5f', index=False)
    del df1
    del df2, df3, df4, df5, df6
#     print('5..')
    gc.collect()

100%|██████████| 54/54 [22:49<00:00, 25.36s/it]


In [11]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    train_org = pd.read_csv(fn_org)
    test_org = pd.read_csv(fn_org1)
    df_shop_all = pd.concat([train_org, test_org], axis=0)
    
#     fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
#     train_org = pd.read_csv(fn_org)
    
    fn = '../cache/df_all_rsdow_lag_shop{}.csv'.format(shop_num)
    df = pd.read_csv(fn)
    df['date'] = df_shop_all.date.values
    
    df_train = df[:len(train_org)]
    df_train = df_train[(df_train.date.astype('str') >= str('2015-01-01')) & 
                        (df_train.date.astype('str') <= str('2017-08-15'))]
    df_train = df_train.drop('date', axis=1)
    
    df_test = df[len(train_org):]
    df_test = df_test.drop('date', axis=1)
    assert(len(test_org) == len(df_test))
    
    fn_tr = '../cache/df_train10_t_rsdow_lag_shop{}.csv'.format(shop_num)
    fn_te = '../cache/df_test7_1_rsdow_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn_tr)
    delete_file_if_exists(fn_te)
    
    df_train.to_csv(fn_tr, index=False, float_format='%.5f')
    df_test.to_csv(fn_te, index=False, float_format='%.5f')
    
    del df_shop_all
    del train_org, test_org
    del df_train
    del df_test

  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  2%|▏         | 1/54 [00:25<22:08, 25.07s/it]

../cache/bkup2/test7_1_t_store2.csv


  4%|▎         | 2/54 [00:53<22:59, 26.53s/it]

...

 96%|█████████▋| 52/54 [19:52<00:45, 22.94s/it]

../cache/bkup2/test7_1_t_store53.csv


 98%|█████████▊| 53/54 [20:12<00:22, 22.88s/it]

../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 54/54 [20:26<00:00, 22.72s/it]


In [ ]:
# Multi-threaded processes to utilize all available CPUs for this task. Note that many threads will block on IO
# so creating more than number of CPUs.    
# thread_pool = Pool(32)
# thread_pool.map (segment_pad_and_save_ct_scan_as_npz, dicom_folder_list)

# # Cleanup
# thread_pool.close()
# thread_pool.join_thread()

In [ ]:
gc.collect()

In [ ]:
# i = 1
# fn = '../cache/train10_t_store{}.csv'.format(i)
# df = pd.read_csv(fn)
# df.unit_sales[df.unit_sales < 0] = 0
# df1 = df.set_index('date')
# dates = np.unique(df.date.values)
# for j, date in tqdm(enumerate(dates)):
#     if j > 0:
#         break
#     print(j)
#     calculate_recent_stats(date, df, df1)
# for days in [15, 30, 60, 90, 180, 365, 730, 1095, 1460]:
#     df = set_recent_df_stats(df ,df1, days)
# print(df.shape)
# print(sum(df.isnull().any()==True))
# print(sum((df != 0).any(axis=0)))
# fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
# print(fn)

In [ ]:
# df.isnull().sum()

In [ ]:
# for i in tqdm(range(44,55)):
#     for dow in range(7):
#         fn_tmp = '../cache/train10_t_store{}_recent_stats_dow{}_tmp.csv'.format(i, dow)
#         delete_file_if_exists(fn_tmp)
#     fn = '../cache/train10_t_store{}_recent_stats_dow.csv'.format(i)
#     df = pd.read_csv(fn)
#     df.to_csv(fn, float_format='%.5f', index=False)
#     del df

In [3]:
from scipy.sparse import save_npz, load_npz
import sys

sys.stdout = open('/dev/stdout', 'w')
new_cols = []

In [4]:
recent_days = [30, 90, 180, 365, 730]
for days in tqdm(recent_days):
    for dow in range(5, 7):
        new_cols.append('us_{}_dow{}_mean'.format(days, dow))
        new_cols.append('us_{}_dow{}_med'.format(days, dow))
        new_cols.append('us_{}_dow{}_std'.format(days, dow))
        new_cols.append('us_{}_dow{}_skew'.format(days, dow))
        new_cols.append('us_{}_dow{}_kurt'.format(days, dow))
        new_cols.append('us_{}_dow{}_tbn'.format(days, dow))

100%|██████████| 5/5 [00:00<00:00, 8709.10it/s]


In [5]:
df_test_rsdow = pd.DataFrame()
for store_num in range(1,55):
    print(store_num)
    fn = '../cache/test7_1_t_store{}_recent_stats_dow_1.csv'.format(store_num)
    df = pd.read_csv(fn)
    df = df[new_cols + ['id']]
    df_test_rsdow = pd.concat([df_test_rsdow, df], axis=0)
    del df

In [6]:
df_test_rsdow = df_test_rsdow.sort_values('id')
df_test_rsdow = df_test_rsdow[new_cols]

In [7]:
df_test_rsdow.shape

(3370464, 60)

In [8]:
delete_file_if_exists('../cache/df_test_rsdow.csv')
df_test_rsdow.to_csv('../cache/df_test_rsdow.csv', index=False, float_format='%.5f')

In [9]:
X_test_rsdow = df_test_rsdow.values
np.save('../cache/X_test_rsdow', X_test_rsdow)